In [1]:
#IMPORTS

import numpy as np
import matplotlib.pyplot as plt
import pygame
import random

from pygame.locals import (
    RLEACCEL,
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)


pygame 2.1.2 (SDL 2.0.16, Python 3.8.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#CLASS DEFINITIONS

class Drone(pygame.sprite.Sprite):
    def __init__(self, ID, xi, yi, zi, mode):
        self.x = xi
        self.y = yi
        self.z = zi
        self.ID = ID
        self.mode = mode
        super(Drone, self).__init__()
        self.surf = pygame.Surface((10, 10))
        self.surf.fill((0, 191, 255))
        self.rect = self.surf.get_rect(
            center=(xi,yi,)
        )

        self.direction = np.pi / 2
        #self.direction = random.randint(0,43) / 7
        self.force = 10

        self.l = self.rect.left
        self.t = self.rect.top
        self.prevl = self.l
        self.prevt = self.t

    def update(self,SCREEN_WIDTH, SCREEN_HEIGHT):

        self.prevl = self.l
        self.prevt = self.t


        if self.mode==0:
            self.direction=0
            self.force=0
        elif self.mode==1:
            self.direction = random.randint(0,8)
            self.force = 10
        elif self.mode==2:
            self.direction = 0
            self.force = 10

        if self.force > 0:
            angle = self.direction*np.pi/4
            dx = round(self.force * np.cos(self.direction))
            dy = -1*round(self.force * np.sin(self.direction))
            self.rect.move_ip(dx,dy)

        if self.rect.left < 0:
            self.rect.left = 0
        if self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top <= 0:
            self.rect.top = 0
        if self.rect.bottom >= SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

        self.l = self.rect.left
        self.t = self.rect.top

class Drone_NSEW(pygame.sprite.Sprite):
    def __init__(self, ID, xi, yi, zi):
        self.x = xi
        self.y = yi
        self.z = zi
        self.ID = ID
        super(Drone_NSEW, self).__init__()
        self.surf = pygame.Surface((10, 10))
        self.surf.fill((0, 191, 255))
        self.rect = self.surf.get_rect(
            center=(xi,yi,)
        )

        self.move = 'r'
        #self.direction = random.randint(0,43) / 7
        self.force = 1

        self.l = self.rect.left
        self.t = self.rect.top
        self.prevl = self.l
        self.prevt = self.t

    def update(self,SCREEN_WIDTH, SCREEN_HEIGHT):

        self.prevl = self.l
        self.prevt = self.t

        if self.force > 0:
            if self.move == 'r':
                self.rect.move_ip(self.force,0)
            elif self.move =='u':
                self.rect.move_ip(0,-1*self.force)
            elif self.move == 'l':
                self.rect.move_ip(-1*self.force,0)
            elif self.move == 'd':
                self.rect.move_ip(0,self.force)

        if self.rect.left < 0:
            self.rect.left = 0
        if self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top <= 0:
            self.rect.top = 0
        if self.rect.bottom >= SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

        self.l = self.rect.left
        self.t = self.rect.top

class Goal(pygame.sprite.Sprite):
    def __init__(self, x, y, z):
        super(Goal, self).__init__()
        self.surf = pygame.Surface((10, 10))
        self.surf.fill((57,255,20))
        self.rect = self.surf.get_rect(
            center=(x,y,)
        )

class Obstacle(pygame.sprite.Sprite):
    def __init__(self,x,y,z,xdim,ydim,zdim):
        super(Obstacle, self).__init__()
        self.surf = pygame.Surface((xdim,ydim))
        self.surf.fill((255,0,0))
        self.rect = self.surf.get_rect(
            center=(x,y,)
        )

class Scanner(pygame.sprite.Sprite):
    def __init__(self,left,top,xdim,ydim,dir):
        super(Scanner, self).__init__()
        self.surf = pygame.Surface((xdim,ydim))
        self.surf.fill((255,255,255))
        self.rect = self.surf.get_rect()
        self.rect.left = left
        self.rect.top = top
        self.dir = dir

    def update(self):
        if self.dir == "r":
            self.rect.move_ip(1,0)
        elif self.dir == "l":
            self.rect.move_ip(-1,0)
        elif self.dir == "u":
            self.rect.move_ip(0,-1)
        elif self.dir == "d":
            self.rect.move_ip(0,1)
        elif self.dir == "ur":
            self.rect.move_ip(1,-1)
        elif self.dir == "ul":
            self.rect.move_ip(-1,-1)
        elif self.dir == "dl":
            self.rect.move_ip(-1,1)
        elif self.dir == "dr":
            self.rect.move_ip(1,1)

        

In [3]:
#OBSERVE() DEFINITION

def observe(entity,spritesgroup,dist,scansgroup,ogroup,ggroup,dgroup):
    l=entity.rect.left
    lx=l-1
    r=entity.rect.right
    t=entity.rect.top
    tx=t-1
    b=entity.rect.bottom

    txx = t-10
    lxx = l-10


    results = np.array([[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]])

    rscan = Scanner(r,t,1,10,"r")
    uscan = Scanner(l,tx,10,1,"u")
    lscan = Scanner(lx,t,1,10,"l")
    dscan = Scanner(l,b,10,1,"d")
    urscan = Scanner(r,txx,10,10,"ur")
    ulscan = Scanner(lxx,txx,10,10,"ul")
    dlscan = Scanner(lxx,b,10,10,"dl")
    drscan = Scanner(r,b,10,10,"dr")
    scansgroup.add(rscan,uscan,lscan,dscan,urscan,ulscan,dlscan,drscan)

    for i in range(dist):
        if pygame.sprite.spritecollideany(rscan,spritesgroup):
            if pygame.sprite.spritecollideany(rscan,ogroup):
                results[0] = [1,i]
            elif pygame.sprite.spritecollideany(rscan,ggroup):
                results[0] = [2,i]
            elif pygame.sprite.spritecollideany(rscan,dgroup):
                results[0] = [3,i]
            break
        rscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(uscan,spritesgroup):
            if pygame.sprite.spritecollideany(uscan,ogroup):
                results[1] = [1,i]
            elif pygame.sprite.spritecollideany(uscan,ggroup):
                results[1] = [2,i]
            elif pygame.sprite.spritecollideany(uscan,dgroup):
                results[1] = [3,i]
            break
        uscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(lscan,spritesgroup):
            if pygame.sprite.spritecollideany(lscan,ogroup):
                results[2] = [1,i]
            elif pygame.sprite.spritecollideany(lscan,ggroup):
                results[2] = [2,i]
            elif pygame.sprite.spritecollideany(lscan,dgroup):
                results[2] = [3,i]
            break
        lscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(dscan,spritesgroup):
            if pygame.sprite.spritecollideany(dscan,ogroup):
                results[3] = [1,i]
            elif pygame.sprite.spritecollideany(dscan,ggroup):
                results[3] = [2,i]
            elif pygame.sprite.spritecollideany(dscan,dgroup):
                results[3] = [3,i]
            break
        dscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(urscan,spritesgroup):
            if pygame.sprite.spritecollideany(urscan,ogroup):
                results[4] = [1,i+19]
            elif pygame.sprite.spritecollideany(urscan,ggroup):
                results[4] = [2,i+19]
            elif pygame.sprite.spritecollideany(urscan,dgroup):
                results[4] = [3,i+19]
            break
        urscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(ulscan,spritesgroup):
            if pygame.sprite.spritecollideany(ulscan,ogroup):
                results[5] = [1,i+19]
            elif pygame.sprite.spritecollideany(ulscan,ggroup):
                results[5] = [2,i+19]
            elif pygame.sprite.spritecollideany(ulscan,dgroup):
                results[5] = [3,i+19]
            break
        ulscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(dlscan,spritesgroup):
            if pygame.sprite.spritecollideany(dlscan,ogroup):
                results[6] = [1,i+19]
            elif pygame.sprite.spritecollideany(dlscan,ggroup):
                results[6] = [2,i+19]
            elif pygame.sprite.spritecollideany(dlscan,dgroup):
                results[6] = [3,i+19]
            break
        dlscan.update()

    for i in range(dist):
        if pygame.sprite.spritecollideany(drscan,spritesgroup):
            if pygame.sprite.spritecollideany(drscan,ogroup):
                results[7] = [1,i+19]
            elif pygame.sprite.spritecollideany(drscan,ggroup):
                results[7] = [2,i+19]
            elif pygame.sprite.spritecollideany(drscan,dgroup):
                results[7] = [3,i+19]
            break
        drscan.update()


    for entity in scansgroup:
        entity.kill()

    return results

In [4]:
#ONESIM() DEFINITION

def onesim(maxlen):
    pygame.init()

    clock = pygame.time.Clock()

    #Sets the height and width of the environment
    SCREEN_WIDTH = 800
    SCREEN_HEIGHT = 600

    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    # output array is num_frames*num_drones*num_coords
    posarr = np.zeros((maxlen,6,3))

    allsprites = pygame.sprite.Group()

    # 6 drones are currently hardcoded in with starting positions
    d1 = Drone(1,np.random.randint(50,100),200,0,1)
    d2 = Drone(2,np.random.randint(50,100),250,0,1)
    d3 = Drone(3,np.random.randint(50,100),300,0,1)
    d4 = Drone(4,np.random.randint(50,100),350,0,2)
    d5 = Drone(5,np.random.randint(50,100),400,0,2)
    d6 = Drone(6,np.random.randint(50,100),450,0,2)

    drones = pygame.sprite.Group()
    drones.add(d1,d2,d3,d4,d5,d6)
    allsprites.add(d1,d2,d3,d4,d5,d6)

    dronelist = [d1,d2,d3,d4,d5,d6]

    # A green 'goal' is randomly placed somehwere on the right side of the environment
    ygoal = np.random.randint(150,450)
    endgoal = Goal(750,ygoal,0)

    goals = pygame.sprite.Group()
    goals.add(endgoal)
    allsprites.add(endgoal)

    goal_l = endgoal.rect.left
    goal_t = endgoal.rect.top

    # Two obstacles randomly placed in the middle of the environment
    yo1 = np.random.randint(150,450)
    xo1 = np.random.randint(250,450)
    o1 = Obstacle(xo1,yo1,0,20,300,0)

    yo2 = np.random.randint(150,450)
    xo2 = np.random.randint(450,650)
    o2 = Obstacle(550,yo2,0,20,300,0)

    obstacles = pygame.sprite.Group()
    obstacles.add(o1,o2)
    allsprites.add(o1,o2)

    # If running is ever false, the run terminates
    running = True
    counter = 0

    scans = pygame.sprite.Group()

    while running:
        # Can force exit the simulation using ESC or by closing the window
        for event in pygame.event.get():
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False

            elif event.type == QUIT:
                running = False

        for entity in drones:
            x = observe(entity,allsprites,100,scans,obstacles,goals,drones)
            #print(x)


        reward = [0,0,0,0,0,0]


        # Drones update, according to the update function defined in Drone class
        #drones.update(SCREEN_WIDTH,SCREEN_HEIGHT)
        for entity in drones:
            prevl = entity.rect.left
            prevt = entity.rect.top
            prevdist = np.sqrt((goal_l-prevl)**2 + (goal_t-prevt)**2)

            entity.update(SCREEN_WIDTH,SCREEN_HEIGHT)
            
            nowl = entity.rect.left
            nowt = entity.rect.top
            nowdist = np.sqrt((goal_l-nowl)**2 + (goal_t-nowt)**2)

            gamma = 0.999

            indireward = gamma**counter * (prevdist-nowdist)

            index=entity.ID - 1

            reward[index] = indireward

            #print(goal_l,goal_t,prevl,prevt,prevdist,nowl,nowt,nowdist,reward)

        #print()



        screen.fill((0,0,0))

        for entity in allsprites:
            screen.blit(entity.surf, entity.rect)
            
        for entity in drones:
            # Drone is killed if it collides with an obstacle
            if pygame.sprite.spritecollideany(entity, obstacles):
                entity.kill()
                reward[entity.ID-1] -= 100

            # Drone is killed if it collides with the goal (success!)
            if pygame.sprite.spritecollideany(entity,goals):
                entity.kill()

            # Both drones are killed if they collide with each other
            for entity2 in drones:
                if entity != entity2 and pygame.sprite.collide_rect(entity, entity2):
                    entity.kill()
                    entity2.kill()
                    reward[entity.ID-1] -= 100
                    reward[entity2.ID-1] -= 100

        #print(reward)
    
        pygame.display.flip()

        dead = 0

        # Store results in posarr
        for i in range(len(dronelist)):
            entity = dronelist[i]
            if entity.alive():
                posarr[counter,i] = [entity.rect.left,entity.rect.top,0]
            else:
                dead += 1
                posarr[counter,i] = [-1,-1,-1]

        # Run ends early if all drones are dead
        if dead == 6:
            running = False

        # Run ends if max length has been reached
        counter += 1
        if counter >= maxlen:
            running = False

        # I'm not sure if this is helping the code run more smoothly or not
        clock.tick(60)

    pygame.quit()
    
    return posarr

In [5]:
#MULTISIM() DEFINITION

def multisim(numsims,maxlen):
    multisimlist = []
    for i in range(numsims):
        x=onesim(maxlen)
        multisimlist.append(x)
    multisimlist=np.array(multisimlist)

    return multisimlist

In [103]:
x=onesim(100)

In [104]:
z=multisim(3,300)

In [6]:
# Calculates the Manhattan distance of each drone from the goal - this is the heuristic for our A* algorithm
def potential(state,goal_l,goal_t):
    p = 0
    for drone in state:
        p += abs(drone[0]-goal_l)
        p += abs(drone[1]-goal_t)
    return p

# So turns out the heuristic technically isn't valid for multiple drones, so I made this one
# (The other one works much better, but isn't technically guaranteed to find the optimal path)
# Effectively, potential() has a tendency to favour states that have made progress towards the goal,
# while newpotential() will search much earlier states in the hope of finding a better route
def newpotential(state,goal_l,goal_t):
    p = 0
    drones = 0
    for drone in state:
        drones += 1
        p += abs(drone[0]-goal_l)
        p += abs(drone[1]-goal_t)
    normp = p/drones
    return normp

In [109]:
#Single drone - code still regularly iterates through the drone list because it will help later

pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

allsprites = pygame.sprite.Group()

# Drone gets a random starting position
d1 = Drone_NSEW(1,np.random.randint(50,100),np.random.randint(250,350),0)

# Initial positions are all stored in case I need to access them in a later notebook cell
dinits = [[d1.rect.left,d1.rect.top]]

drones = pygame.sprite.Group()
drones.add(d1)
allsprites.add(d1)

for entity in drones:
    # sped the code up by getting the drone to move 10 spaces at once rather than 1 - does require a few other numbers in this cell to be tweaked
    entity.force = 10

# probably unneccesary, since the drone doesn't actually crash
dronelist = [d1]

# A green 'goal' is randomly placed somehwere on the right side of the environment
ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)

goal_l = endgoal.rect.left
goal_t = endgoal.rect.top

# Two obstacles randomly placed in the middle of the environment
yo1 = np.random.randint(150,450)
xo1 = np.random.randint(250,450)
o1 = Obstacle(xo1,yo1,0,20,300,0)

yo2 = np.random.randint(150,450)
xo2 = np.random.randint(450,650)
o2 = Obstacle(550,yo2,0,20,300,0)

oinits =[[o1.rect.left,o1.rect.top],[o2.rect.left,o2.rect.top]]

obstacles = pygame.sprite.Group()
obstacles.add(o1,o2)
allsprites.add(o1,o2)

# Each ID corresponds to a given state, and has a source indicating which ID precedes it
states = {}
sources = {}

states[0] = dinits

# Since state 0 is the start position, it gets a weird negative source. Don't question it.
sources[0] = -1

currentstate = dinits
currentdist = 0
currentID = 0


currentpotential = potential(dinits,goal_l,goal_t)

# initial state occupies the first (and only) spot in the queue. The item in a* list is always distance + potential
priorityqueue = [0]
distlist = [0]
potlist = [currentpotential]
astarlist = [currentpotential]

# nothing in this queue... yet
donequeue = []
donedistlist = []
donepotlist = []
doneastarlist = []

# used for assigning new IDs
counter = 1

# pauses the sim to give me time to set my computer recording, feel free to comment out the next 5 lines if they are unnecessary
#screen.fill((0,0,0))
#for entity in allsprites:
#    screen.blit(entity.surf, entity.rect)
#pygame.display.flip()
#pygame.time.wait(10000)

# If running is ever false, the run terminates
running = True

while running:
    # Can force exit the simulation using ESC or by closing the window
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                running = False

        elif event.type == QUIT:
            running = False

    

    commands = [0]


    # 4 possible moves can be made
    for a in ['u','d','r','l']:
        commands[0] = a


        # make sure drone is in the correct position (since it moves to new positions to check them)
        for i in range(len(dronelist)):
            entity = dronelist[i]
            entity.rect.left = currentstate[i][0]
            entity.rect.top = currentstate[i][1]

        # move drone according to commands list
        for i in range(len(dronelist)):
            entity = dronelist[i]
            entity.move = commands[i]
            entity.update(SCREEN_WIDTH, SCREEN_HEIGHT)

        # need to check if position reached is possible - if the drone would crash we will not use this state         
        crash = 0
        for entity in drones:
            # did the drone crash with an obstacle?
            if pygame.sprite.spritecollideany(entity,obstacles):
                crash = 1
            # did the drone crash with another drone?
            else:
                for entity2 in drones:
                    if entity != entity2 and pygame.sprite.collide_rect(entity, entity2):
                        crash = 1
                            
                        
        dronepos = [[d1.rect.left,d1.rect.top]]

        # If the state has already been spotted previously, should check if this is a new, shorter path to the state
        if crash == 0 and dronepos in states.values():
            # Janky piece of code to find the ID of this state
            ID = [k for k, v in states.items() if v == dronepos]

            # The state should REALLY only have one ID - if not something has gone very wrong
            if len(ID) > 1:
                print('You are about to encounter some never before seen errors')
            ID = ID[0]
            # Identify if the state is in the priority queue (not explored yet) or done queue (already explored)
            if ID in priorityqueue and ID in donequeue:
                print('Not good')


            elif ID in priorityqueue:
                pos = priorityqueue.index(ID)
                newdist = currentdist + 10
                # if the original path to the state is shorter (or equal to) the new one there is no need to do anything
                # otherwise, we need to move it to a new spot in the priority queue
                if newdist < distlist[pos]:
                    # use pop to extract and remove the relevant info from the queues
                    x = priorityqueue.pop(pos)
                    y = distlist.pop(pos)
                    y = currentdist + 10
                    z = potlist.pop(pos)
                    w = astarlist.pop(pos)
                    w = y + z

                    # change source node to reflect new shortest path
                    sources[ID] = currentID

                    # Find the right spot in the priority queue to add the state (based on the astar value)
                    if w >= astarlist[-1]:
                        priorityqueue.append(x)
                        distlist.append(y)
                        potlist.append(z)
                        astarlist.append(w)
                    else:
                        for i in range(1,len(astarlist)):
                            if w <= astarlist[i]:
                                index=i
                                priorityqueue.insert(index,x)
                                distlist.insert(index,y)
                                potlist.insert(index,z)
                                astarlist.insert(index,w)
                                break


            # basically the exact same thing as above, but for items in the done queue
            elif ID in donequeue:
                pos = donequeue.index(ID)
                newdist = currentdist + 10
                if newdist < donedistlist[pos]:
                    x = donequeue.pop(pos)
                    y = donedistlist.pop(pos)
                    y = currentdist + 10
                    z = donepotlist.pop(pos)
                    w = doneastarlist.pop(pos)
                    w = y + z

                    sources[ID] = currentID

                    if w >= astarlist[-1]:
                        priorityqueue.append(x)
                        distlist.append(y)
                        potlist.append(z)
                        astarlist.append(w)
                    else:
                        for i in range(1,len(astarlist)):
                            if w <= astarlist[i]:
                                index=i
                                priorityqueue.insert(index,x)
                                distlist.insert(index,y)
                                potlist.insert(index,z)
                                astarlist.insert(index,w)
                                break

            # it should be in one of the queues - if not, something is wrong
            else:
                print('Not good')
            

        # what to do if the state hasn't been seen before
        if crash == 0 and dronepos not in states.values():
            
            # add a new state with a new ID
            states[counter] = dronepos
            sources[counter] = currentID

            # each 'state change' costs 10, since it moves the drone 10 spaces
            dist = currentdist + 10
            pot = potential(dronepos,goal_l,goal_t)
            astar = dist+pot
            
            # same as above, finds the correct spot in the priority queue to add the new state
            if astar >= astarlist[-1]:
                priorityqueue.append(counter)
                distlist.append(dist)
                potlist.append(pot)
                astarlist.append(astar)
            else:
                for i in range(1,len(astarlist)):
                    if astar <= astarlist[i]:
                        index=i
                        priorityqueue.insert(index,counter)
                        distlist.insert(index,dist)
                        potlist.insert(index,pot)
                        astarlist.insert(index,astar)
                        break

            # counter increases, since we need a new ID to assign to the next new state we find
            counter += 1

        # This resets the drones to the current state - it might be unnecessary now since I started doing that earlier
        for i in range(len(dronelist)):
            entity = dronelist[i]
            entity.rect.left = currentstate[i][0]
            entity.rect.top = currentstate[i][1]


    # Updates the screen - try containing it in a for loop if you want the code to run faster (maybe twice as fast? IDK) enclose it in an if statement
    # if counter % 100 == 0:
    screen.fill((0,0,0))
    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)
    pygame.display.flip()

    # Move the explored element (the first in the priority queue) to the end of the donequeue
    x = priorityqueue.pop(0)
    y = distlist.pop(0)
    z = potlist.pop(0)
    w = astarlist.pop(0)

    donequeue.append(x)
    donedistlist.append(y)
    donepotlist.append(z)
    doneastarlist.append(w)

    # re-allocate the first element of the priority queue as the new state to explore
    currentID = priorityqueue[0]
    currentstate = states[currentID]
    currentdist = distlist[0]
    currentpotential = potlist[0]

    # Once potential is low enough, the drone has got close to the goal and our work is done
    if currentpotential <= 10:
        running=False

    # Useful if you want to test the algorithm without waiting for an entire run to finish
    #if counter >= 1000:
    #    running=False

# If the code is interrupted, Pygame will not quit, and you will need to run some more code to close the Pygame window
pygame.quit()
    
#print(dinits)
#print(goal_l,goal_t)
#print(oinits)
#print(len(priorityqueue))
#print(donequeue)

[[66, 284]]
745 260
[[382, 0], [540, 247]]


In [110]:
# Simple piece of code - takes the last state in the done queue (the end state) and works backwards to get the order of states taken
# to reach it

shortestpath = []
nowstate = priorityqueue[0]
shortestpath.append(nowstate)
while True:
    prevstate = sources[nowstate]
    if prevstate == -1:
        break
    else:
        nowstate = prevstate
        shortestpath.append(nowstate)

print(shortestpath)

[670, 667, 665, 663, 660, 657, 654, 651, 648, 645, 642, 639, 636, 633, 630, 627, 624, 621, 618, 615, 612, 610, 608, 606, 503, 403, 342, 340, 338, 337, 335, 334, 331, 328, 325, 322, 319, 316, 313, 310, 307, 304, 301, 298, 296, 294, 292, 222, 93, 91, 88, 85, 82, 79, 76, 73, 70, 67, 64, 61, 58, 55, 52, 49, 46, 43, 40, 37, 34, 31, 28, 25, 22, 19, 16, 13, 10, 7, 3, 0]


In [111]:
# Shows you what the shortest path looks like
pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

allsprites = pygame.sprite.Group()

d1 = Drone_NSEW(1,50,300,0)

# initial drone position is set according to dinits
d1.rect.left = dinits[0][0]
d1.rect.top = dinits[0][1]

drones = pygame.sprite.Group()
drones.add(d1)
allsprites.add(d1)

dronelist = [d1]

endgoal = Goal(750,300,0)
# goal position set according to goal_l and goal_t
endgoal.rect.left = goal_l
endgoal.rect.top = goal_t

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)

# Two obstacles placed according to oinits
o1 = Obstacle(300,300,0,20,300,0)
o1.rect.left = oinits[0][0]
o1.rect.top = oinits[0][1]

o2 = Obstacle(550,300,0,20,300,0)
o2.rect.left = oinits[1][0]
o2.rect.top = oinits[1][1]

obstacles = pygame.sprite.Group()
obstacles.add(o1,o2)
allsprites.add(o1,o2)


# Uncomment lines below if you want a pause before the simulation (?) starts
#screen.fill((0,0,0))
#for entity in allsprites:
#    screen.blit(entity.surf, entity.rect)
#pygame.display.flip()
#pygame.time.wait(10000)

for i in range(len(shortestpath)):

    # The shortest path list is written backwards, sorry!
    thisstate = shortestpath[-1*(i+1)]
    dronepos = states[thisstate]

    # Move the drone into the designated position
    for i in range(len(dronelist)):
        entity = dronelist[i]
        entity.rect.left = dronepos[i][0]
        entity.rect.top = dronepos[i][1]


    # VISUALISE
    screen.fill((0,0,0))
    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)
    pygame.display.flip()

pygame.quit()

In [7]:
# For multiple drones, we apply a penalty if they fly in an undesired formation
# In this version, drones are punished if they are too far apart or too close together

def goodformation(dronepos):
    penalty = 0
    for i in range(len(dronepos)):
        for j in range(i+1,len(dronepos)):
            dist = abs(dronepos[i][0]-dronepos[j][0]) + abs(dronepos[i][1]-dronepos[j][1])
            if dist >= 50:
                penalty += dist
            elif dist <= 20:
                penalty += 500/(dist+1)

    return penalty

In [27]:
# 2 drones, penalised if they move too far apart
# Code is mostly the same as the one drone example, I've only commented the new bits

pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

allsprites = pygame.sprite.Group()

# 2 drones now!
d1 = Drone_NSEW(1,np.random.randint(50,100),np.random.randint(265,285),0)
d2 = Drone_NSEW(1,np.random.randint(50,100),np.random.randint(315,335),0)

dinits = [[d1.rect.left,d1.rect.top],[d2.rect.left,d2.rect.top]]

drones = pygame.sprite.Group()
drones.add(d1,d2)
allsprites.add(d1,d2)

for entity in drones:
    entity.force = 10

dronelist = [d1,d2]

ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)

goal_l = endgoal.rect.left
goal_t = endgoal.rect.top

yo1 = np.random.randint(150,450)
xo1 = np.random.randint(250,450)
o1 = Obstacle(xo1,yo1,0,20,300,0)

yo2 = np.random.randint(150,450)
xo2 = np.random.randint(450,650)
o2 = Obstacle(550,yo2,0,20,300,0)

oinits =((o1.rect.left,o1.rect.top),(o2.rect.left,o2.rect.top))

obstacles = pygame.sprite.Group()
obstacles.add(o1,o2)
allsprites.add(o1,o2)


running = True

states = {}
sources = {}

states[0] = dinits
sources[0] = -1

currentstate = dinits
currentdist = 0
currentID = 0


currentpotential = potential(dinits,goal_l,goal_t)
priorityqueue = [0]
distlist = [0]
potlist = [currentpotential]
astarlist = [currentpotential]

donequeue = []
donedistlist = []
donepotlist = []
doneastarlist = []

counter = 1

#screen.fill((0,0,0))
#for entity in allsprites:
#    screen.blit(entity.surf, entity.rect)
#pygame.display.flip()
#pygame.time.wait(10000)

while running:
    # Can force exit the simulation using ESC or by closing the window
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                running = False

        elif event.type == QUIT:
            running = False

    
    # 2 commands now, and I've nexted a second for loop
    commands = [0,0]

    for a in ['u','d','r','l']:
        commands[0] = a
        for b in ['u','d','r','l']:
            commands[1] = b


            for i in range(len(dronelist)):
                entity = dronelist[i]
                entity.rect.left = currentstate[i][0]
                entity.rect.top = currentstate[i][1]
            
            for i in range(len(dronelist)):
                entity = dronelist[i]
                entity.move = commands[i]
                entity.update(SCREEN_WIDTH, SCREEN_HEIGHT)
                            
            crash = 0
            for entity in drones:
                if pygame.sprite.spritecollideany(entity,obstacles):
                    crash = 1
                else:
                    for entity2 in drones:
                        if entity != entity2 and pygame.sprite.collide_rect(entity, entity2):
                            crash = 1
                            
                        

            dronepos = [[d1.rect.left,d1.rect.top],[d2.rect.left,d2.rect.top]]

            # Calculate the penalty for the new position
            penalty = goodformation(dronepos)

            if crash == 0 and dronepos in states.values():
                ID = [k for k, v in states.items() if v == dronepos]
                if len(ID) > 1:
                    print('You are about to encounter some never before seen errors')
                ID = ID[0]
                if ID in priorityqueue and ID in donequeue:
                    print('Not good')


                elif ID in priorityqueue:
                    pos = priorityqueue.index(ID)
                    newdist = currentdist + 10 + penalty
                    if newdist < distlist[pos]:
                        x = priorityqueue.pop(pos)
                        y = distlist.pop(pos)
                        y = currentdist + 10 + penalty
                        z = potlist.pop(pos)
                        w = astarlist.pop(pos)
                        w = y + z

                        sources[ID] = currentID

                        if w >= astarlist[-1]:
                            priorityqueue.append(x)
                            distlist.append(y)
                            potlist.append(z)
                            astarlist.append(w)
                        else:
                            for i in range(1,len(astarlist)):
                                if w <= astarlist[i]:
                                    index=i
                                    priorityqueue.insert(index,x)
                                    distlist.insert(index,y)
                                    potlist.insert(index,z)
                                    astarlist.insert(index,w)
                                    break



                elif ID in donequeue:
                    pos = donequeue.index(ID)
                    newdist = currentdist + 10 + penalty
                    if newdist < donedistlist[pos]:
                        x = donequeue.pop(pos)
                        y = donedistlist.pop(pos)
                        y = currentdist + 10 + penalty
                        z = donepotlist.pop(pos)
                        w = doneastarlist.pop(pos)
                        w = y + z

                        sources[ID] = currentID

                        if w >= astarlist[-1]:
                            priorityqueue.append(x)
                            distlist.append(y)
                            potlist.append(z)
                            astarlist.append(w)
                        else:
                            for i in range(1,len(astarlist)):
                                if w <= astarlist[i]:
                                    index=i
                                    priorityqueue.insert(index,x)
                                    distlist.insert(index,y)
                                    potlist.insert(index,z)
                                    astarlist.insert(index,w)
                                    break


                else:
                    print('Not good')
            

            if crash == 0 and dronepos not in states.values():
                states[counter] = dronepos
                sources[counter] = currentID
                dist = currentdist + 10 + penalty
                pot = potential(dronepos,goal_l,goal_t)
                astar = dist+pot
                if astar >= astarlist[-1]:
                    priorityqueue.append(counter)
                    distlist.append(dist)
                    potlist.append(pot)
                    astarlist.append(astar)
                else:
                    for i in range(1,len(astarlist)):
                        if astar <= astarlist[i]:
                            index=i
                            priorityqueue.insert(index,counter)
                            distlist.insert(index,dist)
                            potlist.insert(index,pot)
                            astarlist.insert(index,astar)
                            break

                counter += 1

            for i in range(len(dronelist)):
                entity = dronelist[i]
                entity.rect.left = currentstate[i][0]
                entity.rect.top = currentstate[i][1]


    if counter % 100 == 0:
        screen.fill((0,0,0))

        for entity in allsprites:
            screen.blit(entity.surf, entity.rect)
    
        pygame.display.flip()

    x = priorityqueue.pop(0)
    y = distlist.pop(0)
    z = potlist.pop(0)
    w = astarlist.pop(0)

    donequeue.append(x)
    donedistlist.append(y)
    donepotlist.append(z)
    doneastarlist.append(w)

    currentID = priorityqueue[0]
    currentstate = states[currentID]
    currentdist = distlist[0]
    currentpotential = potlist[0]

    if currentpotential <= 100:
        running=False

    #if counter >= 100000:
    #    running=False

pygame.quit()
    
#print(dinits)
#print(goal_l,goal_t)
#print(oinits)
#print(len(priorityqueue))
#print(donequeue)

[[67, 274], [50, 318]]
745 204
((267, 74), (540, 31))


In [28]:
# This is unchanged from the one drone example. I should probably make it into a function

shortestpath = []
nowstate = priorityqueue[0]
shortestpath.append(nowstate)
while True:
    prevstate = sources[nowstate]
    if prevstate == -1:
        break
    else:
        nowstate = prevstate
        shortestpath.append(nowstate)

print(shortestpath)

[32233, 32218, 32203, 32188, 32173, 32158, 32143, 32128, 32113, 32099, 32084, 32077, 32062, 32048, 32033, 32020, 32005, 31990, 31975, 31960, 31945, 31930, 31915, 31900, 31885, 31870, 31855, 31840, 31825, 31810, 31795, 31780, 31765, 31750, 31735, 31720, 31705, 31690, 31675, 31660, 31645, 31630, 31615, 31600, 31585, 31570, 31555, 31540, 31525, 31510, 31495, 31480, 31465, 31450, 31435, 31420, 31405, 31395, 31385, 28763, 25700, 23691, 23685, 23678, 17741, 12789, 8752, 5415, 2873, 303, 294, 279, 264, 249, 234, 219, 204, 189, 174, 159, 144, 129, 114, 99, 84, 69, 54, 40, 27, 15, 0]


In [29]:
pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

allsprites = pygame.sprite.Group()

# 2 drones now
d1 = Drone_NSEW(1,np.random.randint(50,100),300,0)
d2 = Drone_NSEW(2,np.random.randint(50,100),250,0)

d1.rect.left = dinits[0][0]
d1.rect.top = dinits[0][1]
d2.rect.left = dinits[1][0]
d2.rect.top = dinits[1][1]

drones = pygame.sprite.Group()
drones.add(d1,d2)
allsprites.add(d1,d2)

dronelist = [d1,d2]


ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)
endgoal.rect.left = goal_l
endgoal.rect.top = goal_t

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)


yo1 = np.random.randint(150,450)
xo1 = np.random.randint(250,450)
o1 = Obstacle(xo1,yo1,0,20,300,0)
o1.rect.left = oinits[0][0]
o1.rect.top = oinits[0][1]

yo2 = np.random.randint(150,450)
xo2 = np.random.randint(450,650)
o2 = Obstacle(550,yo2,0,20,300,0)
o2.rect.left = oinits[1][0]
o2.rect.top = oinits[1][1]

obstacles = pygame.sprite.Group()
obstacles.add(o1,o2)
allsprites.add(o1,o2)



#screen.fill((0,0,0))
#for entity in allsprites:
#    screen.blit(entity.surf, entity.rect)
#pygame.display.flip()
#pygame.time.wait(10000)

for i in range(len(shortestpath)):

    thisstate = shortestpath[-1*(i+1)]
    dronepos = states[thisstate]

    for i in range(len(dronelist)):
        entity = dronelist[i]
        entity.rect.left = dronepos[i][0]
        entity.rect.top = dronepos[i][1]

    screen.fill((0,0,0))

    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)
    
    pygame.display.flip()

pygame.quit()

In [9]:
# Same 2 drone set up as previously, but now with some funkier obstacles

pygame.init()

clock = pygame.time.Clock()

SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

allsprites = pygame.sprite.Group()

d1 = Drone_NSEW(1,np.random.randint(50,100),np.random.randint(265,285),0)
d2 = Drone_NSEW(1,np.random.randint(50,100),np.random.randint(315,335),0)

dinits = [[d1.rect.left,d1.rect.top],[d2.rect.left,d2.rect.top]]

drones = pygame.sprite.Group()
drones.add(d1,d2)
allsprites.add(d1,d2)

for entity in drones:
    entity.force = 10

dronelist = [d1,d2]

ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)

goal_l = endgoal.rect.left
goal_t = endgoal.rect.top

# Now we make lots of little rectangles for obstacles, and spread them about randomly
# You could imagine it as a bunch of skyscrapers in a city that our drones have to dodge
obstacles = pygame.sprite.Group()
oinits = []

for i in range(75):
    y = np.random.randint(50,550)
    x = np.random.randint(250,650)
    ydim = np.random.randint(10,30)
    xdim = np.random.randint(10,30)
    obs = Obstacle(x,y,0,xdim,ydim,0)
    obstacles.add(obs)
    allsprites.add(obs)
    oinits.append([obs.rect.left,obs.rect.top])


# If running is ever false, the run terminates
running = True

states = {}
sources = {}

states[0] = dinits
sources[0] = -1

currentstate = dinits
currentdist = 0
currentID = 0


currentpotential = potential(dinits,goal_l,goal_t)
priorityqueue = [0]
distlist = [0]
potlist = [currentpotential]
astarlist = [currentpotential]

donequeue = []
donedistlist = []
donepotlist = []
doneastarlist = []

counter = 1

screen.fill((0,0,0))
for entity in allsprites:
    screen.blit(entity.surf, entity.rect)
pygame.display.flip()
pygame.time.wait(10000)

while running:
    # Can force exit the simulation using ESC or by closing the window
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                running = False

        elif event.type == QUIT:
            running = False

    

    commands = [0,0]

    for a in ['u','d','r','l']:
        commands[0] = a
        for b in ['u','d','r','l']:
            commands[1] = b


            for i in range(len(dronelist)):
                entity = dronelist[i]
                entity.rect.left = currentstate[i][0]
                entity.rect.top = currentstate[i][1]
            
            for i in range(len(dronelist)):
                entity = dronelist[i]
                entity.move = commands[i]
                entity.update(SCREEN_WIDTH, SCREEN_HEIGHT)
                            
            crash = 0
            for entity in drones:
                if pygame.sprite.spritecollideany(entity,obstacles):
                    crash = 1
                else:
                    for entity2 in drones:
                        if entity != entity2 and pygame.sprite.collide_rect(entity, entity2):
                            crash = 1
                            
                        

            dronepos = [[d1.rect.left,d1.rect.top],[d2.rect.left,d2.rect.top]]

            penalty = goodformation(dronepos)

            if crash == 0 and dronepos in states.values():
                ID = [k for k, v in states.items() if v == dronepos]
                if len(ID) > 1:
                    print('You are about to encounter some never before seen errors')
                ID = ID[0]
                if ID in priorityqueue and ID in donequeue:
                    print('Not good')


                elif ID in priorityqueue:
                    pos = priorityqueue.index(ID)
                    newdist = currentdist + 10 + penalty
                    if newdist < distlist[pos]:
                        x = priorityqueue.pop(pos)
                        y = distlist.pop(pos)
                        y = currentdist + 10 + penalty
                        z = potlist.pop(pos)
                        w = astarlist.pop(pos)
                        w = y + z

                        sources[ID] = currentID

                        if w >= astarlist[-1]:
                            priorityqueue.append(x)
                            distlist.append(y)
                            potlist.append(z)
                            astarlist.append(w)
                        else:
                            for i in range(1,len(astarlist)):
                                if w <= astarlist[i]:
                                    index=i
                                    priorityqueue.insert(index,x)
                                    distlist.insert(index,y)
                                    potlist.insert(index,z)
                                    astarlist.insert(index,w)
                                    break



                elif ID in donequeue:
                    pos = donequeue.index(ID)
                    newdist = currentdist + 10 + penalty
                    if newdist < donedistlist[pos]:
                        x = donequeue.pop(pos)
                        y = donedistlist.pop(pos)
                        y = currentdist + 10 + penalty
                        z = donepotlist.pop(pos)
                        w = doneastarlist.pop(pos)
                        w = y + z

                        sources[ID] = currentID

                        if w >= astarlist[-1]:
                            priorityqueue.append(x)
                            distlist.append(y)
                            potlist.append(z)
                            astarlist.append(w)
                        else:
                            for i in range(1,len(astarlist)):
                                if w <= astarlist[i]:
                                    index=i
                                    priorityqueue.insert(index,x)
                                    distlist.insert(index,y)
                                    potlist.insert(index,z)
                                    astarlist.insert(index,w)
                                    break


                else:
                    print('Not good')
            

            if crash == 0 and dronepos not in states.values():
                states[counter] = dronepos
                sources[counter] = currentID
                dist = currentdist + 10 + penalty
                pot = potential(dronepos,goal_l,goal_t)
                astar = dist+pot
                if astar >= astarlist[-1]:
                    priorityqueue.append(counter)
                    distlist.append(dist)
                    potlist.append(pot)
                    astarlist.append(astar)
                else:
                    for i in range(1,len(astarlist)):
                        if astar <= astarlist[i]:
                            index=i
                            priorityqueue.insert(index,counter)
                            distlist.insert(index,dist)
                            potlist.insert(index,pot)
                            astarlist.insert(index,astar)
                            break

                counter += 1

            for i in range(len(dronelist)):
                entity = dronelist[i]
                entity.rect.left = currentstate[i][0]
                entity.rect.top = currentstate[i][1]


    
    screen.fill((0,0,0))

    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)
    
    pygame.display.flip()

    x = priorityqueue.pop(0)
    y = distlist.pop(0)
    z = potlist.pop(0)
    w = astarlist.pop(0)

    donequeue.append(x)
    donedistlist.append(y)
    donepotlist.append(z)
    doneastarlist.append(w)

    currentID = priorityqueue[0]
    currentstate = states[currentID]
    currentdist = distlist[0]
    currentpotential = potlist[0]

    if currentpotential <= 100:
        running=False

    #if counter >= 100000:
    #    running=False

pygame.quit()
    
#print(dinits)
#print(goal_l,goal_t)
#print(oinits)
#print(len(priorityqueue))
#print(donequeue)

In [121]:
shortestpath = []
nowstate = priorityqueue[0]
shortestpath.append(nowstate)
while True:
    prevstate = sources[nowstate]
    if prevstate == -1:
        break
    else:
        nowstate = prevstate
        shortestpath.append(nowstate)

print(shortestpath)

[1172, 1157, 1142, 1127, 1112, 1097, 1082, 1067, 1056, 1045, 1034, 1026, 1015, 1009, 1001, 995, 976, 948, 943, 935, 928, 918, 911, 891, 885, 879, 863, 829, 822, 805, 791, 776, 761, 746, 731, 716, 701, 686, 671, 656, 641, 626, 611, 596, 581, 566, 551, 536, 521, 506, 491, 481, 469, 453, 442, 433, 428, 356, 347, 340, 333, 323, 293, 282, 267, 252, 237, 222, 207, 192, 177, 162, 147, 132, 117, 102, 87, 72, 57, 42, 27, 11, 0]


In [127]:
pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

allsprites = pygame.sprite.Group()

d1 = Drone_NSEW(1,np.random.randint(50,100),300,0)
d2 = Drone_NSEW(2,np.random.randint(50,100),250,0)

d1.rect.left = dinits[0][0]
d1.rect.top = dinits[0][1]
d2.rect.left = dinits[1][0]
d2.rect.top = dinits[1][1]

drones = pygame.sprite.Group()
drones.add(d1,d2)
allsprites.add(d1,d2)

dronelist = [d1,d2]

ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)
endgoal.rect.left = goal_l
endgoal.rect.top = goal_t

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)


for entity in obstacles:
    allsprites.add(entity)


#screen.fill((0,0,0))
#for entity in allsprites:
#    screen.blit(entity.surf, entity.rect)
#pygame.display.flip()
#pygame.time.wait(10000)

for i in range(len(shortestpath)):

    thisstate = shortestpath[-1*(i+1)]
    dronepos = states[thisstate]

    for i in range(len(dronelist)):
        entity = dronelist[i]
        entity.rect.left = dronepos[i][0]
        entity.rect.top = dronepos[i][1]

    screen.fill((0,0,0))

    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)
    
    pygame.display.flip()

pygame.quit()

In [10]:
# Slight modification to the penalty created earlier

def goodformation2(dronepos):
    penalty = 0
    for i in range(len(dronepos)):
        for j in range(i+1,len(dronepos)):
            dist = abs(dronepos[i][0]-dronepos[j][0]) + abs(dronepos[i][1]-dronepos[j][1])
            if dist >= 35:
                penalty += dist
            elif dist <= 20:
                penalty += 500/(dist+1)

    return penalty

In [11]:
# In this variant, drones are penalise when they don't fly in a horizontal line
# Also punished if they get too close to each other

def goodformation3(dronepos):
    penalty = 0
    for i in range(len(dronepos)):
        for j in range(i+1,len(dronepos)):
            ydiff=abs(dronepos[i][1]-dronepos[j][1])
            if ydiff >= 5:
                penalty += ydiff
            
            dist = abs(dronepos[i][0]-dronepos[j][0]) + abs(dronepos[i][1]-dronepos[j][1])
            if dist <= 20:
                penalty += 500/(dist+1)

    return penalty

In [12]:
# Same as above, but a vertical line

def goodformation4(dronepos):
    penalty = 0
    for i in range(len(dronepos)):
        for j in range(i+1,len(dronepos)):
            xdiff=abs(dronepos[i][0]-dronepos[j][0])
            if xdiff >= 5:
                penalty += xdiff
            
            dist = abs(dronepos[i][0]-dronepos[j][0]) + abs(dronepos[i][1]-dronepos[j][1])
            if dist <= 20:
                penalty += 500/(dist+1)

    return penalty

In [16]:
# 3 drones?!?!

pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))


allsprites = pygame.sprite.Group()

d1 = Drone_NSEW(1,np.random.randint(50,100),np.random.randint(250,270),0)
d2 = Drone_NSEW(1,np.random.randint(50,100),np.random.randint(290,310),0)
d3 = Drone_NSEW(1,np.random.randint(50,100),np.random.randint(330,350),0)

#d1 = Drone_NSEW(1,75,275,0)
#d2 = Drone_NSEW(1,75,300,0)
#d3 = Drone_NSEW(1,75,325,0)

dinits = [[d1.rect.left,d1.rect.top],[d2.rect.left,d2.rect.top],[d3.rect.left,d3.rect.top]]

drones = pygame.sprite.Group()
drones.add(d1,d2,d3)
allsprites.add(d1,d2,d3)

for entity in drones:
    entity.force = 10

dronelist = [d1,d2,d3]

ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)

goal_l = endgoal.rect.left
goal_t = endgoal.rect.top


obstacles = pygame.sprite.Group()
oinits = []

for i in range(50):
    y = np.random.randint(50,550)
    x = np.random.randint(250,650)
    ydim = np.random.randint(10,25)
    xdim = np.random.randint(10,25)
    obs = Obstacle(x,y,0,xdim,ydim,0)
    obstacles.add(obs)
    allsprites.add(obs)
    oinits.append([obs.rect.left,obs.rect.top])



running = True

states = {}
sources = {}

states[0] = dinits
sources[0] = -1

currentstate = dinits
currentdist = 0
currentID = 0


currentpotential = potential(dinits,goal_l,goal_t)
priorityqueue = [0]
distlist = [0]
potlist = [currentpotential]
astarlist = [currentpotential]

donequeue = []
donedistlist = []
donepotlist = []
doneastarlist = []

counter = 1

#screen.fill((0,0,0))
#for entity in allsprites:
#    screen.blit(entity.surf, entity.rect)
#pygame.display.flip()
#pygame.time.wait(10000)

while running:
    # Can force exit the simulation using ESC or by closing the window
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                running = False

        elif event.type == QUIT:
            running = False

    

    commands = [0,0,0]

    # I gotta find a better way to do this
    for a in ['u','d','r','l']:
        commands[0] = a
        for b in ['u','d','r','l']:
            commands[1] = b
            for c in ['u','d','r','l']:
                commands[2] = c


                for i in range(len(dronelist)):
                    entity = dronelist[i]
                    entity.rect.left = currentstate[i][0]
                    entity.rect.top = currentstate[i][1]
            
                for i in range(len(dronelist)):
                    entity = dronelist[i]
                    entity.move = commands[i]
                    entity.update(SCREEN_WIDTH, SCREEN_HEIGHT)
                            
                crash = 0
                for entity in drones:
                    if pygame.sprite.spritecollideany(entity,obstacles):
                        crash = 1
                    else:
                        for entity2 in drones:
                            if entity != entity2 and pygame.sprite.collide_rect(entity, entity2):
                                crash = 1
                            
                        

                dronepos = [[d1.rect.left,d1.rect.top],[d2.rect.left,d2.rect.top],[d3.rect.left,d3.rect.top]]

                penalty = goodformation4(dronepos)

                if crash == 0 and dronepos in states.values():
                    ID = [k for k, v in states.items() if v == dronepos]
                    if len(ID) > 1:
                        print('You are about to encounter some never before seen errors')
                    ID = ID[0]
                    if ID in priorityqueue and ID in donequeue:
                        print('Not good')


                    elif ID in priorityqueue:
                        pos = priorityqueue.index(ID)
                        newdist = currentdist + 10 + penalty
                        if newdist < distlist[pos]:
                            x = priorityqueue.pop(pos)
                            y = distlist.pop(pos)
                            y = currentdist + 10 + penalty
                            z = potlist.pop(pos)
                            w = astarlist.pop(pos)
                            w = y + z

                            sources[ID] = currentID

                            if w >= astarlist[-1]:
                                priorityqueue.append(x)
                                distlist.append(y)
                                potlist.append(z)
                                astarlist.append(w)
                            else:
                                for i in range(1,len(astarlist)):
                                    if w <= astarlist[i]:
                                        index=i
                                        priorityqueue.insert(index,x)
                                        distlist.insert(index,y)
                                        potlist.insert(index,z)
                                        astarlist.insert(index,w)
                                        break



                    elif ID in donequeue:
                        pos = donequeue.index(ID)
                        newdist = currentdist + 10 + penalty
                        if newdist < donedistlist[pos]:
                            x = donequeue.pop(pos)
                            y = donedistlist.pop(pos)
                            y = currentdist + 10 + penalty
                            z = donepotlist.pop(pos)
                            w = doneastarlist.pop(pos)
                            w = y + z

                            sources[ID] = currentID

                            if w >= astarlist[-1]:
                                priorityqueue.append(x)
                                distlist.append(y)
                                potlist.append(z)
                                astarlist.append(w)
                            else:
                                for i in range(1,len(astarlist)):
                                    if w <= astarlist[i]:
                                        index=i
                                        priorityqueue.insert(index,x)
                                        distlist.insert(index,y)
                                        potlist.insert(index,z)
                                        astarlist.insert(index,w)
                                        break


                    else:
                        print('Not good')
            

                if crash == 0 and dronepos not in states.values():
                    states[counter] = dronepos
                    sources[counter] = currentID
                    dist = currentdist + 10 + penalty
                    pot = potential(dronepos,goal_l,goal_t)
                    astar = dist+pot
                    if astar >= astarlist[-1]:
                        priorityqueue.append(counter)
                        distlist.append(dist)
                        potlist.append(pot)
                        astarlist.append(astar)
                    else:
                        for i in range(1,len(astarlist)):
                            if astar <= astarlist[i]:
                                index=i
                                priorityqueue.insert(index,counter)
                                distlist.insert(index,dist)
                                potlist.insert(index,pot)
                                astarlist.insert(index,astar)
                                break

                    counter += 1

                for i in range(len(dronelist)):
                    entity = dronelist[i]
                    entity.rect.left = currentstate[i][0]
                    entity.rect.top = currentstate[i][1]


    screen.fill((0,0,0))
    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)  
    pygame.display.flip()

    x = priorityqueue.pop(0)
    y = distlist.pop(0)
    z = potlist.pop(0)
    w = astarlist.pop(0)

    donequeue.append(x)
    donedistlist.append(y)
    donepotlist.append(z)
    doneastarlist.append(w)

    currentID = priorityqueue[0]
    currentstate = states[currentID]
    currentdist = distlist[0]
    currentpotential = potlist[0]

    if currentpotential <= 150:
        running=False

    #if counter >= 100000:
    #    running=False

pygame.quit()
    
#print(dinits)
#print(goal_l,goal_t)
#print(oinits)
#print(len(priorityqueue))
#print(donequeue)

In [14]:
shortestpath = []
nowstate = priorityqueue[0]
shortestpath.append(nowstate)
while True:
    prevstate = sources[nowstate]
    if prevstate == -1:
        break
    else:
        nowstate = prevstate
        shortestpath.append(nowstate)

print(shortestpath)

[4924, 4901, 4842, 4783, 4724, 4665, 4606, 4547, 4488, 4429, 4375, 4337, 4320, 4228, 4200, 4116, 4091, 3937, 3914, 3879, 3835, 3516, 3456, 3391, 3338, 3313, 3258, 3203, 3152, 3116, 3087, 3058, 3033, 2866, 2396, 2382, 2264, 2257, 2219, 2180, 2151, 2122, 2053, 1998, 1958, 1911, 1884, 1865, 1840, 1712, 1672, 1629, 1583, 1532, 1481, 1445, 1409, 1376, 1339, 1247, 1206, 1166, 1134, 1095, 1064, 1036, 1003, 970, 953, 897, 862, 807, 752, 697, 642, 587, 532, 477, 422, 367, 312, 257, 202, 147, 92, 36, 0]


In [15]:
pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

allsprites = pygame.sprite.Group()

d1 = Drone_NSEW(1,np.random.randint(50,100),300,0)
d2 = Drone_NSEW(2,np.random.randint(50,100),250,0)
d3 = Drone_NSEW(3,np.random.randint(50,100),350,0)

d1.rect.left = dinits[0][0]
d1.rect.top = dinits[0][1]
d2.rect.left = dinits[1][0]
d2.rect.top = dinits[1][1]
d3.rect.left = dinits[2][0]
d3.rect.top = dinits[2][1]

drones = pygame.sprite.Group()
drones.add(d1,d2,d3)
allsprites.add(d1,d2,d3)

dronelist = [d1,d2,d3]

ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)
endgoal.rect.left = goal_l
endgoal.rect.top = goal_t

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)


for entity in obstacles:
    allsprites.add(entity)


screen.fill((0,0,0))
for entity in allsprites:
    screen.blit(entity.surf, entity.rect)
pygame.display.flip()
pygame.time.wait(10000)

for i in range(len(shortestpath)):

    thisstate = shortestpath[-1*(i+1)]
    dronepos = states[thisstate]

    for i in range(len(dronelist)):
        entity = dronelist[i]
        entity.rect.left = dronepos[i][0]
        entity.rect.top = dronepos[i][1]

    screen.fill((0,0,0))

    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)
    
    pygame.display.flip()

pygame.quit()

In [79]:
# Another slight variation on the penalty function
# It is quite important for a successful sim that it is possible for the drones to get 0 penalty somehow

def goodformation5(dronepos):
    penalty = 0
    for i in range(len(dronepos)):
        for j in range(i+1,len(dronepos)):
            dist = abs(dronepos[i][0]-dronepos[j][0]) + abs(dronepos[i][1]-dronepos[j][1])
            if dist >= 70:
                penalty += dist
            elif dist <= 20:
                penalty += 500/(dist+1)

    return penalty

In [93]:
# 4 drones?!?!

pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

allsprites = pygame.sprite.Group()

#d1 = Drone_NSEW(1,np.random.randint(50,100),np.random.randint(230,250),0)
#d2 = Drone_NSEW(1,np.random.randint(50,100),np.random.randint(270,290),0)
#d3 = Drone_NSEW(1,np.random.randint(50,100),np.random.randint(310,330),0)
#d4 = Drone_NSEW(1,np.random.randint(50,100),np.random.randint(350,370),0)


# The drones are often not very good at getting into formation if their starting position is bad, so I added square and horizontal line
# starting positions
#d1 = Drone_NSEW(1,50,310,0)
#d2 = Drone_NSEW(1,50,285,0)
#d3 = Drone_NSEW(1,75,310,0)
#d4 = Drone_NSEW(1,75,285,0)

d1 = Drone_NSEW(1,np.random.randint(10,30),np.random.randint(275,325),0)
d2 = Drone_NSEW(1,np.random.randint(50,70),np.random.randint(275,325),0)
d3 = Drone_NSEW(1,np.random.randint(90,110),np.random.randint(275,325),0)
d4 = Drone_NSEW(1,np.random.randint(130,150),np.random.randint(275,325),0)

dinits = [[d1.rect.left,d1.rect.top],[d2.rect.left,d2.rect.top],[d3.rect.left,d3.rect.top],[d4.rect.left,d4.rect.top]]

drones = pygame.sprite.Group()
drones.add(d1,d2,d3,d4)
allsprites.add(d1,d2,d3,d4)

for entity in drones:
    entity.force = 10

dronelist = [d1,d2,d3,d4]

ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)

goal_l = endgoal.rect.left
goal_t = endgoal.rect.top

obstacles = pygame.sprite.Group()
oinits = []

for i in range(50):
    y = np.random.randint(50,550)
    x = np.random.randint(250,650)
    ydim = np.random.randint(10,25)
    xdim = np.random.randint(10,30)
    obs = Obstacle(x,y,0,xdim,ydim,0)
    obstacles.add(obs)
    allsprites.add(obs)
    oinits.append([obs.rect.left,obs.rect.top])

running = True

states = {}
sources = {}

states[0] = dinits
sources[0] = -1

currentstate = dinits
currentdist = 0
currentID = 0


currentpotential = potential(dinits,goal_l,goal_t)
priorityqueue = [0]
distlist = [0]
potlist = [currentpotential]
astarlist = [currentpotential]

donequeue = []
donedistlist = []
donepotlist = []
doneastarlist = []

counter = 1

#screen.fill((0,0,0))
#for entity in allsprites:
#    screen.blit(entity.surf, entity.rect)
#pygame.display.flip()
#pygame.time.wait(10000)

while running:
    # Can force exit the simulation using ESC or by closing the window
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                running = False

        elif event.type == QUIT:
            running = False

    

    commands = [0,0,0,0]

    for a in ['u','d','r','l']:
        commands[0] = a
        for b in ['u','d','r','l']:
            commands[1] = b
            for c in ['u','d','r','l']:
                commands[2] = c
                for d in ['u','d','r','l']:
                    commands[3] = d


                    for i in range(len(dronelist)):
                        entity = dronelist[i]
                        entity.rect.left = currentstate[i][0]
                        entity.rect.top = currentstate[i][1]
                
                    for i in range(len(dronelist)):
                        entity = dronelist[i]
                        entity.move = commands[i]
                        entity.update(SCREEN_WIDTH, SCREEN_HEIGHT)
                                
                    crash = 0
                    for entity in drones:
                        if pygame.sprite.spritecollideany(entity,obstacles):
                            crash = 1
                        else:
                            for entity2 in drones:
                                if entity != entity2 and pygame.sprite.collide_rect(entity, entity2):
                                    crash = 1
                                
                            

                    dronepos = [[d1.rect.left,d1.rect.top],[d2.rect.left,d2.rect.top],[d3.rect.left,d3.rect.top],[d4.rect.left,d4.rect.top]]

                    penalty = goodformation3(dronepos)

                    if crash == 0 and dronepos in states.values():
                        ID = [k for k, v in states.items() if v == dronepos]
                        if len(ID) > 1:
                            print('You are about to encounter some never before seen errors')
                        ID = ID[0]
                        if ID in priorityqueue and ID in donequeue:
                            print('Not good')


                        elif ID in priorityqueue:
                            pos = priorityqueue.index(ID)
                            newdist = currentdist + 10 + penalty
                            if newdist < distlist[pos]:
                                x = priorityqueue.pop(pos)
                                y = distlist.pop(pos)
                                y = currentdist + 10 + penalty
                                z = potlist.pop(pos)
                                w = astarlist.pop(pos)
                                w = y + z

                                sources[ID] = currentID

                                if w >= astarlist[-1]:
                                    priorityqueue.append(x)
                                    distlist.append(y)
                                    potlist.append(z)
                                    astarlist.append(w)
                                else:
                                    for i in range(1,len(astarlist)):
                                        if w <= astarlist[i]:
                                            index=i
                                            priorityqueue.insert(index,x)
                                            distlist.insert(index,y)
                                            potlist.insert(index,z)
                                            astarlist.insert(index,w)
                                            break



                        elif ID in donequeue:
                            pos = donequeue.index(ID)
                            newdist = currentdist + 10 + penalty
                            if newdist < donedistlist[pos]:
                                x = donequeue.pop(pos)
                                y = donedistlist.pop(pos)
                                y = currentdist + 10 + penalty
                                z = donepotlist.pop(pos)
                                w = doneastarlist.pop(pos)
                                w = y + z

                                sources[ID] = currentID

                                if w >= astarlist[-1]:
                                    priorityqueue.append(x)
                                    distlist.append(y)
                                    potlist.append(z)
                                    astarlist.append(w)
                                else:
                                    for i in range(1,len(astarlist)):
                                        if w <= astarlist[i]:
                                            index=i
                                            priorityqueue.insert(index,x)
                                            distlist.insert(index,y)
                                            potlist.insert(index,z)
                                            astarlist.insert(index,w)
                                            break


                        else:
                            print('Not good')
                

                    if crash == 0 and dronepos not in states.values():
                        states[counter] = dronepos
                        sources[counter] = currentID
                        dist = currentdist + 10 + penalty
                        pot = potential(dronepos,goal_l,goal_t)
                        astar = dist+pot
                        if astar >= astarlist[-1]:
                            priorityqueue.append(counter)
                            distlist.append(dist)
                            potlist.append(pot)
                            astarlist.append(astar)
                        else:
                            for i in range(1,len(astarlist)):
                                if astar <= astarlist[i]:
                                    index=i
                                    priorityqueue.insert(index,counter)
                                    distlist.insert(index,dist)
                                    potlist.insert(index,pot)
                                    astarlist.insert(index,astar)
                                    break

                        counter += 1

                    for i in range(len(dronelist)):
                        entity = dronelist[i]
                        entity.rect.left = currentstate[i][0]
                        entity.rect.top = currentstate[i][1]


    
    screen.fill((0,0,0))

    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)
    
    pygame.display.flip()

    x = priorityqueue.pop(0)
    y = distlist.pop(0)
    z = potlist.pop(0)
    w = astarlist.pop(0)

    donequeue.append(x)
    donedistlist.append(y)
    donepotlist.append(z)
    doneastarlist.append(w)

    currentID = priorityqueue[0]
    currentstate = states[currentID]
    currentdist = distlist[0]
    currentpotential = potlist[0]

    if currentpotential <= 120:
        running=False

    #if counter >= 100000:
    #    running=False

pygame.quit()
    
#print(dinits)
#print(goal_l,goal_t)
#print(oinits)
#print(len(priorityqueue))
#print(donequeue)

[[15, 295], [53, 293], [91, 277], [136, 306]]
745 233


In [90]:
shortestpath = []
nowstate = priorityqueue[0]
shortestpath.append(nowstate)
while True:
    prevstate = sources[nowstate]
    if prevstate == -1:
        break
    else:
        nowstate = prevstate
        shortestpath.append(nowstate)

print(shortestpath)

[21938, 21577, 20389, 19254, 18439, 17991, 16233, 15692, 15520, 15167, 14943, 14704, 14625, 14386, 14207, 14028, 13789, 13550, 13326, 13147, 13014, 12868, 12577, 12484, 12229, 11990, 11815, 11663, 11479, 11247, 11211, 11068, 11011, 10996, 10840, 10075, 9906, 9860, 9784, 9452, 9251, 9146, 8955, 8776, 8581, 8358, 8163, 7924, 7685, 7446, 7207, 6968, 6729, 6554, 6379, 6140, 5901, 5662, 5451, 5320, 5177, 4982, 4759, 4581, 4391, 4048, 3977, 3755, 3576, 3397, 3202, 2979, 2784, 2545, 2306, 2067, 1828, 1589, 1350, 1112, 874, 615, 135, 0]


In [92]:
pygame.init()

clock = pygame.time.Clock()

#Sets the height and width of the environment
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

allsprites = pygame.sprite.Group()

d1 = Drone_NSEW(1,np.random.randint(50,100),300,0)
d2 = Drone_NSEW(2,np.random.randint(50,100),250,0)
d3 = Drone_NSEW(3,np.random.randint(50,100),350,0)
d4 = Drone_NSEW(4,np.random.randint(50,100),400,0)

d1.rect.left = dinits[0][0]
d1.rect.top = dinits[0][1]
d2.rect.left = dinits[1][0]
d2.rect.top = dinits[1][1]
d3.rect.left = dinits[2][0]
d3.rect.top = dinits[2][1]
d4.rect.left = dinits[3][0]
d4.rect.top = dinits[3][1]

drones = pygame.sprite.Group()
drones.add(d1,d2,d3,d4)
allsprites.add(d1,d2,d3,d4)

dronelist = [d1,d2,d3,d4]

ygoal = np.random.randint(150,450)
endgoal = Goal(750,ygoal,0)
endgoal.rect.left = goal_l
endgoal.rect.top = goal_t

goals = pygame.sprite.Group()
goals.add(endgoal)
allsprites.add(endgoal)


for entity in obstacles:
    allsprites.add(entity)


#screen.fill((0,0,0))
#for entity in allsprites:
#    screen.blit(entity.surf, entity.rect)
#pygame.display.flip()
#pygame.time.wait(10000)

for i in range(len(shortestpath)):

    thisstate = shortestpath[-1*(i+1)]
    dronepos = states[thisstate]

    for i in range(len(dronelist)):
        entity = dronelist[i]
        entity.rect.left = dronepos[i][0]
        entity.rect.top = dronepos[i][1]

    screen.fill((0,0,0))

    for entity in allsprites:
        screen.blit(entity.surf, entity.rect)
    
    pygame.display.flip()

pygame.quit()